# Ingest drivers.json file

## Pre-requisites
- Add data source and file date parameter widgets to the notebook and get it as a variable
- Run configuration notebook with folder paths
- Run common_functions notebook

In [0]:
dbutils.widgets.text("param_data_source", "")
var_data_source = dbutils.widgets.get("param_data_source")

In [0]:
dbutils.widgets.text("param_file_date", "")
var_file_date = dbutils.widgets.get("param_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Add ingestion date column to each data frame using current timestamp

### Step 1 - Read the JSON file using the Spark DataFrame Reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
name_schema = StructType(fields = [
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True),
])

In [0]:
drivers_schema = StructType(fields = [
    StructField("driverId", IntegerType(), False),
    StructField("driverRef", StringType(), True),
    StructField("number", IntegerType(), True),
    StructField("code", StringType(), True),
    StructField("name", name_schema),
    StructField("dob", DateType(), True),
    StructField("nationality", StringType(), True),
    StructField("url", StringType(), True)
])

In [0]:
drivers_df = spark.read \
    .schema(drivers_schema) \
    .json(f'{raw_folder_path}/{var_file_date}/drivers.json')

## Step 2 - Rename columns and add ingestion date and concatenated name columns

In [0]:
from pyspark.sql.functions import col, concat, lit

In [0]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
                                    .withColumn("data_source", lit(var_data_source)) \
                                    .withColumn("file_date", lit(var_file_date))

In [0]:
drivers_with_ingestion_date_df = add_ingestion_date(drivers_with_columns_df)

## Step 3 - Drop unwanted columns

In [0]:
drivers_final_df = drivers_with_ingestion_date_df.drop(drivers_with_columns_df['url'])

## Step 4 - Write data to Data Lake as a managed table in Delta Lake format

In [0]:
drivers_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.drivers")

In [0]:
dbutils.notebook.exit("Success")